In [ ]:
!pip install transformers lime hf_xet datasets captum numpy

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader
#from transformers import AutoTokenizer, AutoModelForCausalLM
from lime.lime_text import LimeTextExplainer
from datasets import load_dataset
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
import warnings
import tqdm as tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig
from captum.attr import IntegratedGradients
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
warnings.filterwarnings("ignore")


In [ ]:
dataset = load_dataset("hatexplain")

README.md:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

hatexplain.py:   0%|          | 0.00/4.78k [00:00<?, ?B/s]

The repository for hatexplain contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hatexplain.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/15383 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1922 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1924 [00:00<?, ? examples/s]

In [ ]:
dataset['train'][0]

{'id': '23107796_gab',
 'annotators': {'label': [0, 2, 2],
  'annotator_id': [203, 204, 233],
  'target': [['Hindu', 'Islam'],
   ['Hindu', 'Islam'],
   ['Hindu', 'Islam', 'Other']]},
 'rationales': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]],
 'post_tokens': ['u',
  'really',
  'think',
  'i',
  'would',
  'not',
  'have',
  'been',
  'raped',
  'by',
  'feral',
  'hindu',
  'or',
  'muslim',
  'back',
  'in',
  'india',
  'or',
  'bangladesh',
  'and',
  'a',
  'neo',
  'nazi',
  'would',
  'rape',
  'me',
  'as',
  'well',
  'just',
  'to',
  'see',
  'me',
  'cry']}

In [ ]:
def join_tokens(example):
    example["text"] = " ".join(example["post_tokens"])
    return example

dataset = dataset.map(join_tokens)

dataset["train"][0]["text"]

Map:   0%|          | 0/15383 [00:00<?, ? examples/s]

Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

Map:   0%|          | 0/1924 [00:00<?, ? examples/s]

'u really think i would not have been raped by feral hindu or muslim back in india or bangladesh and a neo nazi would rape me as well just to see me cry'

In [ ]:
def combine_rationale_masks(rationale_masks, method='weighted'):
    """
    Combines multiple binary rationale masks into a single mask.
    """
    rationale_array = np.array(rationale_masks)

    # Union method: If any annotator highlights a token, it is part of the rationale
    if method == 'union':
        final_rationale_mask = (rationale_array.sum(axis=0) > 0).astype(int)

    # Majority method: A token is part of the rationale if the majority of annotators highlight it
    elif method == 'majority':
        final_rationale_mask = (rationale_array.sum(axis=0) >= (len(rationale_masks) / 2)).astype(int)

    # Weighted method: Tokens are weighted based on how many annotators highlight it
    elif method == 'weighted':
        # Compute the average weight for each token (0-1 scale)
        weights = rationale_array.mean(axis=0)
        # Map average values to 0 or 1, with values < 1 being fractional
        final_rationale_mask = weights.tolist()

    else:
        raise ValueError("Unknown method. Choose from 'union', 'majority', or 'weighted'.")

    return final_rationale_mask

def safe_add_final_rationale(example):
    rationales = example.get('rationales', None)

    if not rationales or not isinstance(rationales, list) or not all(isinstance(r, list) for r in rationales):
        # Default to a zero mask if invalid or missing
        num_tokens = len(example.get('post_tokens', []))
        example['final_rationale_mask'] = [0] * num_tokens
        return example

    try:
        example['final_rationale_mask'] = combine_rationale_masks(rationales, method='weighted')
    except Exception as e:
        # Handle edge cases or shape mismatch
        num_tokens = len(example.get('post_tokens', []))
        example['final_rationale_mask'] = [0] * num_tokens

    return example

dataset = dataset.map(safe_add_final_rationale)

Map:   0%|          | 0/15383 [00:00<?, ? examples/s]

Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

Map:   0%|          | 0/1924 [00:00<?, ? examples/s]

In [ ]:
dataset['train'][17]

{'id': '1177990839177490432_twitter',
 'annotators': {'label': [0, 0, 0],
  'annotator_id': [6, 25, 10],
  'target': [['African'],
   ['Disability', 'African', 'Homosexual'],
   ['Women', 'Disability', 'African', 'Homosexual']]},
 'rationales': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]],
 'post_tokens': ['nigger',
  'faggot',
  'retard',
  'beaner',
  'dishwasher',
  'time',
  'to',
  'see',
  'if',
  'i',
  'get',
  'ban'],
 'text': 'nigger faggot retard beaner dishwasher time to see if i get ban',
 'final_rationale_mask': [1.0,
  1.0,
  1.0,
  1.0,
  0.3333333333333333,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]}

In [ ]:
from collections import Counter

def resolve_hate_label(label_list):
    """
    Convert multi-class label (0: hate, 1: neither, 2: offensive) to binary label.
    Returns 'hate' if majority is 0 or 2; otherwise 'non-hate'.
    """
    if not label_list or len(label_list) != 3:
        return "non-detrimental"  # fallback
    counts = Counter(label_list)
    majority_label = counts.most_common(1)[0][0]
    return "detrimenal_content" if majority_label in [0, 2] else "non-detrimental"
def add_binary_label(example):
    example['binary_label'] = resolve_hate_label(example['annotators']['label'])
    return example

dataset = dataset.map(add_binary_label)

Map:   0%|          | 0/15383 [00:00<?, ? examples/s]

Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

Map:   0%|          | 0/1924 [00:00<?, ? examples/s]

In [ ]:
dataset['train'][17]

{'id': '1177990839177490432_twitter',
 'annotators': {'label': [0, 0, 0],
  'annotator_id': [6, 25, 10],
  'target': [['African'],
   ['Disability', 'African', 'Homosexual'],
   ['Women', 'Disability', 'African', 'Homosexual']]},
 'rationales': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]],
 'post_tokens': ['nigger',
  'faggot',
  'retard',
  'beaner',
  'dishwasher',
  'time',
  'to',
  'see',
  'if',
  'i',
  'get',
  'ban'],
 'text': 'nigger faggot retard beaner dishwasher time to see if i get ban',
 'final_rationale_mask': [1.0,
  1.0,
  1.0,
  1.0,
  0.3333333333333333,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'binary_label': 'detrimenal_content'}

In [ ]:
from torch.utils.data import Dataset

class HateXplainDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128):
        """
        dataset: HuggingFace-style Dataset object (already mapped with final_rationale_mask)
        tokenizer: Tokenizer (e.g., from transformers) to tokenize 'text'
        max_length: Max token length for model inputs
        """
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_map = {"non_detrimenal": 0, "detrimental": 1}  # adjust if needed

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]

        # Tokenize text
        encoding = self.tokenizer(
            item['text'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Convert label to binary
        label = 1 if item['binary_label'] == 'detrimenal_content' else 0

        # Final rationale mask (assumes already aligned with post_tokens)
        rationale = item.get('final_rationale_mask', [])

        # Pad rationale to max_length
        rationale_tensor = torch.zeros(self.max_length)
        rationale_values = rationale[:self.max_length]
        rationale_tensor[:len(rationale_values)] = torch.tensor(rationale_values, dtype=torch.float32)

        return {
            'input_ids': encoding['input_ids'].squeeze(0),  # [max_length]
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.float32),
            'rationale_mask': rationale_tensor
        }

In [ ]:
import torch
import torch.nn as nn
from transformers import RobertaModel, RobertaConfig

class CustomRobertaClassifier(nn.Module):
    def __init__(self, pretrained_model_name, hidden_size=128, dropout_rate=0.3):
        super(CustomRobertaClassifier, self).__init__()

        self.roberta = RobertaModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(dropout_rate)

        # Intermediate dense layer
        self.dense = nn.Linear(self.roberta.config.hidden_size, hidden_size)
        self.relu = nn.ReLU()

        # Final classification layer (outputting 1 value for binary classification)
        self.classifier = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [batch_size, hidden_size]

        x = self.dropout(pooled_output)
        x = self.relu(self.dense(x))
        x = self.dropout(x)
        logits = self.classifier(x)

        return self.sigmoid(logits)





In [ ]:


# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


model = CustomRobertaClassifier(pretrained_model_name='roberta-base')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataset = HateXplainDataset(dataset['train'], tokenizer)
validation_dataset = HateXplainDataset(dataset['validation'], tokenizer)
test_dataset = HateXplainDataset(dataset['test'], tokenizer)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [ ]:
def evaluate(model, dataloader, device='cuda'):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            preds = (outputs.squeeze() > 0.5).int()

            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().int().tolist())

    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    print(f"Validation Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")
    return f1

In [ ]:
from captum.attr import LayerIntegratedGradients
import math
def forward_func(single_id, single_mask):
    model.eval()

    input_id = single_id.long()
    attention = single_mask.long()

    out=model(input_id, attention)


    return out
def train(model, train_loader, val_loader, epochs=3, lr=2e-5, lambda_reg=0.5, device='cuda'):
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.BCELoss()
    ig = LayerIntegratedGradients(forward_func, model.roberta.embeddings)
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        prev_val_f1 = 0.5
        loop =tqdm(train_loader, desc=f"Epoch {epoch+1} Training")
        for batch in loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].unsqueeze(1).to(device)
            rationale_mask = batch['rationale_mask'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)

            pred_loss = criterion(outputs, labels)
            if math.isnan(pred_loss.item()):
              print('NAAAAN')
            # Integrated Gradients (token-level importance)


            for i in range(len(labels)):  # Loop over each sample in the batch
                if labels[i].item() == 1:  # Check if the label for this example is class 1
                    # Integrated Gradients (token-level importance)
                    model.zero_grad()

                    single_id = input_ids[i].unsqueeze(0)
                    single_mask = attention_mask[i].unsqueeze(0)

                    attributions = ig.attribute(single_id, additional_forward_args = single_mask, target=0, n_steps=15).sum(dim=-1)
                    attributions = attributions.clamp(min=0)
                    norm_attributions = attributions / (attributions.sum(dim=1, keepdim=True) + 1e-8)
                    if attributions.sum(dim=1, keepdim=True) == 0:
                      continue
                    if rationale_mask[i].sum().item() == 0:
                      continue

                    # KL Divergence (rationales vs. IG importance)

                    target_rationale = rationale_mask[i] / (rationale_mask[i].sum(dim=0, keepdim=True) + 1e-8)
                    target_rationale = target_rationale + 1e-8
                    kl_div = (target_rationale * (target_rationale / (norm_attributions[0] + 1e-8)).log()).sum(dim=0).mean()

                    if kl_div.isnan():
                      print(target_rationale)
                      print(norm_attributions)
                      print('NAAAN')
                    # Add KL Divergence regularization term to the loss
                    pred_loss += lambda_reg * kl_div





            pred_loss.backward()
            optimizer.step()

            total_loss += pred_loss.item()
            loop.set_postfix(loss=pred_loss / 8)
        print(f"Epoch {epoch+1} Training Loss: {total_loss / len(train_loader):.4f}")
        f1 = evaluate(model, val_loader, device)
        if f1 > prev_val_f1:
          prev_val_f1 = f1
          torch.save(model.state_dict(), f'hatexplain_roberta_weights_IG_{round(f1,4)}.pth')

In [ ]:
train(model, train_loader, val_loader, epochs=5, lr=2e-5, lambda_reg=5)
#0.5 - 8215  #0.01 8193 #1

lambda_reg: 0.1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1 Training: 100%|██████████| 1923/1923 [40:55<00:00,  1.28s/it, loss=tensor(0.2749, device='cuda:0', grad_fn=<DivBackward0>)]


Epoch 1 Training Loss: 3.8462


Evaluating: 100%|██████████| 241/241 [00:13<00:00, 17.89it/s]


Validation Accuracy: 0.7529, F1 Score: 0.8138


...

Validation Accuracy: 0.7955, F1 Score: 0.8335


Epoch 3 Training: 100%|██████████| 1923/1923 [40:51<00:00,  1.27s/it, loss=tensor(10.6032, device='cuda:0', grad_fn=<DivBackward0>)]


Epoch 3 Training Loss: 154.4686


Evaluating: 100%|██████████| 241/241 [00:13<00:00, 18.02it/s]


Validation Accuracy: 0.7680, F1 Score: 0.8131


In [ ]:
torch.save(model.state_dict(), 'hatexplain_roberta_weights_IG.pth')

In [ ]:
!zip -r files.zip . -x "*.ipynb_checkpoints*"

  adding: .config/ (stored 0%)
  adding: .config/.last_opt_in_prompt.yaml (stored 0%)
  adding: .config/config_sentinel (stored 0%)
  adding: .config/active_config (stored 0%)
  adding: .config/gce (stored 0%)
  adding: .config/.last_update_check.json (deflated 23%)
  adding: .config/default_configs.db (deflated 98%)
  adding: .config/.last_survey_prompt.yaml (stored 0%)
  adding: .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
  adding: .config/logs/ (stored 0%)
  adding: .config/logs/2025.04.16/ (stored 0%)
  adding: .config/logs/2025.04.16/13.39.57.291033.log (deflated 86%)
  adding: .config/logs/2025.04.16/13.40.10.795692.log (deflated 56%)
  adding: .config/logs/2025.04.16/13.40.01.202474.log (deflated 58%)
  adding: .config/logs/2025.04.16/13.39.18.062772.log (deflated 92%)
  adding: .config/logs/2025.04.16/13.40.10.090744.log (deflated 57%)
  adding: .config/logs/2025.04.16/13.39.48.364195.log (deflated 58%)
  adding: .config/configurations/

In [ ]:
from google.colab import files
files.download("files.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>